# Pending actions I

You can really improve the user experience of your bot by asking them simple yes or no questions. One easy way to handle these follow-ups is to define pending actions which get executed as soon as the user says "yes", and wiped if the user says "no".

In this exercise, you're going to define a policy function which takes the intent as it's sole argument, and returns two values: The next action to take, and a pending action. The policy function should return this value when a "yes" intent is returned, and should wipe the pending actions if a "no" intent is returned.

Here, the interpret(message) function has been defined for you such that if "yes" is in the message, "affirm" is returned, and if "no" is in the message, then "deny" is returned.

In [ ]:
# Define policy()
def policy(intent):
    # Return "do_pending" if the intent is "affirm"
    if intent == "affirm":
        return "do_pending", None
    # Return "Ok" if the intent is "deny"
    if intent == "deny":
        return "Ok", None
    if intent == "order":
        return "Unfortunately, the Kenyan coffee is currently out of stock, would you like to order the Brazilian beans?", "Alright, I've ordered that for you!"


# Pending actions II
Having defined your policy function, it's now time to write a send_message() function which takes both a pending action and a message as its arguments and leverages the policy function to determine the bot's response.

Your policy(intent) function from the previous exercise has been pre-loaded.

In [ ]:
# Define send_message()
def send_message(pending, message):
    print("USER : {}".format(message))
    action, pending_action = policy(interpret(message))
    if action == 'do_pending' and pending is not None:
        print("BOT : {}".format(pending))
    else:
        print("BOT : {}".format(action))
    return pending_action
    
# Define send_messages()
def send_messages(messages):
    pending = None
    for msg in messages:
        pending = send_message(pending, msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "ok yes please"
])

USER : I'd like to order some coffee

BOT : Unfortunately, the Kenyan coffee is currently out of stock, would you like to order the Brazilian beans?

USER : ok yes please

BOT : Alright, I've ordered that for you!


# Pending state transitions
you'll often need to briefly deviate from a flow, for example to authenticate a user, before returning.

In these cases, it's often simpler - and easier to debug - to save some actions/states as pending rather than adding ever more complicated rules.

Here, you're going to define a policy_rules dictionary, where the keys are tuples of the current state and the received intent, while the values are tuples of the next state, the bot's response, and a state for which to set a pending transition.

In [ ]:
def send_message(state, pending, message):
    print("USER : {}".format(message))
    new_state, response, pending_state = policy_rules[(state, interpret(message))]
    print("BOT : {}".format(response))
    if pending is not None:
        new_state, response, pending_state = policy_rules[pending]
        print("BOT : {}".format(response))        
    if pending_state is not None:
        pending = (pending_state, interpret(message))
    return new_state, pending

In [ ]:
# Define the states
INIT=0
AUTHED=1
CHOOSE_COFFEE=2
ORDERED=3

# Define the policy rules
policy_rules = {
    (INIT, "order"): (INIT, "you'll have to log in first, what's your phone number?", AUTHED),
    (INIT, "number"): (AUTHED, "perfect, welcome back!", None),
    (AUTHED, "order"): (CHOOSE_COFFEE, "would you like Columbian or Kenyan?", None),    
    (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!", None)
}

# Define send_messages()
def send_messages(messages):
    state = INIT
    pending = None
    for msg in messages:
        state, pending = send_message(state, pending, msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "555-12345",
    "kenyan"
])


USER : I'd like to order some coffee

BOT : you'll have to log in first, what's your phone number?

USER : 555-12345

BOT : perfect, welcome back!

BOT : would you like Columbian or Kenyan?

USER : kenyan

BOT : perfect, the beans are on their way!

BOT : would you like Columbian or Kenyan?



# Putting it all together I

It's time to put everything together everything you've learned in the course by combining the coffee ordering bot with the eliza rules from chapter 1.

To begin, you'll define a function called chitchat_response(), which calls the predefined function match_rule() from back in chapter 1. This returns a response if the message matched an eliza template, and otherwise, None.

The eliza rules are contained in a dictionary called eliza_rules.

In [10]:
# Define chitchat_response()
def chitchat_response(message):
    # Call match_rule()
    response, phrase = match_rule(eliza_rules, message)
    # Return none is response is "default"
    if response == "default":
        return None
    if '{0}' in response:
        # Replace the pronouns of phrase
        phrase = replace_pronouns(phrase)
        # Calculate the response
        response = response.format(phrase)
    return response


# Putting it all together II
With your chitchat_response(message) function defined, the next step is to define a send_message() function which first calls chitchat_response(message), and only uses the coffee bot policy if there is no matching message.

In [15]:
import re
def match_rule(rules, message):
    for pattern, responses in rules.items():
        match = re.search(pattern, message)
        if match is not None:
            response = random.choice(responses)
            var = match.group(1) if '{0}' in response else None
            return response, var
    return "default", None

In [16]:
eliza_rules = {'do you remember (.*)': ['Did you think I would forget {0}', "Why haven't you been able to forget {0}", 'What about {0}', 'Yes .. and?'], 'I want (.*)': ['What would it mean if you got {0}', 'Why do you want {0}', "What's stopping you from getting {0}"], 'do you think (.*)': ['if {0}? Absolutely.', 'No chance'], 'if (.*)': ["Do you really think it's likely that {0}", 'Do you wish that {0}', 'What do you think about {0}', 'Really--if {0}']}

In [ ]:
# Define send_message()
def send_message(state, pending, message):
    print("USER : {}".format(message))
    response = chitchat_response(message)
    if response is not None:
        print("BOT : {}".format(response))
        return state, None
    
    # Calculate the new_state, response, and pending_state
    new_state, response, pending_state = policy_rules[(state, interpret(message))]
    print("BOT : {}".format(response))
    if pending is not None:
        new_state, response, pending_state = policy_rules[pending]
        print("BOT : {}".format(response))        
    if pending_state is not None:
        pending = (pending_state, interpret(message))
    return new_state, pending

# Define send_messages()
def send_messages(messages):
    state = INIT
    pending = None
    for msg in messages:
        state, pending = send_message(state, pending, msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "555-12345",
    "do you remember when I ordered 1000 kilos by accident?",
    "kenyan"
])  

USER : I'd like to order some coffee

BOT : you'll have to log in first, what's your phone number?

USER : 555-12345

BOT : perfect, welcome back!

BOT : would you like Columbian or Kenyan?

USER : do you remember when I ordered 1000 kilos by accident?

BOT : Yes .. and?

USER : kenyan

BOT : perfect, the beans are on their way!
